# Exercícios

In [1]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']
df.describe()

,nome,descricao,categoria,texto
count,2916,2916,2916,2916
unique,2584,2460,4,2646
top,Boneco Dragon Ball Z Son Gokou,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro,Boneco Dragon Ball Z Son Gokou Produto novo
freq,20,39,838,20


In [2]:
# em caso de ter falha na hora de importar o spacy
# import os

# os.environ['KMP_DUPLICATE_LIB_OK']='True'

## ToDo 1

Use a função criada no exercício da Aula 2 que lematiza uma palavra desde que ela seja um verbo. Crie uma nova coluna chamada texto_lemma que conterá o resultado da aplicação da função. 

In [3]:
#!python -m spacy download pt_core_news_sm

In [4]:
# resposta
import spacy
nlp = spacy.load('pt_core_news_sm')
def lemmatizer(text):
    doc = nlp(text)
    return " ".join([word.lemma_ if word.pos_ == "VERB" else word.text for word in doc])
    
df['texto_lemma'] = df['texto'].apply(lemmatizer)
df.head()

,nome,descricao,categoria,texto,texto_lemma
0,O Hobbit - 7ª Ed. 2013,Produto NovoBilbo Bolseiro é um hobbit que lev...,livro,O Hobbit - 7ª Ed. 2013 Produto NovoBilbo Bol...,O Hobbit - 7ª Ed . 2013 Produto NovoBilbo ...
1,Livro - It A Coisa - Stephen King,Produto NovoDurante as férias escolares de 195...,livro,Livro - It A Coisa - Stephen King Produto No...,Livro - It A Coisa - Stephen King Produto ...
2,Box As Crônicas De Gelo E Fogo Pocket 5 Li...,Produto NovoTodo o reino de Westeros ao alcanc...,livro,Box As Crônicas De Gelo E Fogo Pocket 5 Li...,Box As Crônicas De Gelo E Fogo Pocket ...
3,Box Harry Potter,Produto Novo e Físico A série Harry Potter ch...,livro,Box Harry Potter Produto Novo e Físico A sé...,Box Harry Potter Produto Novo e Físico A...
4,Livro Origem - Dan Brown,Produto NovoDe Onde Viemos? Para Onde Vamos? R...,livro,Livro Origem - Dan Brown Produto NovoDe Onde...,Livro Origem - Dan Brown Produto NovoDe On...


## ToDo 2
Divida o dataset em conjunto de treino e teste e faça a vetorização dos dados usando CountVectorizer. Remova as stop-words usando o conjunto do NLTK. 

In [5]:
#resposta
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df,
      test_size = 0.3, 
      random_state = 42
  )

In [6]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.texto_lemma)
text_vect_train = vect.transform(df_train.texto_lemma)

## ToDo 3

Use a implementação da SKlearn do LDA para treinar um modelo de tópicos. Use os dados obtidos no ToDo 2 como entrada. 

Documentação do LDA na Scikit-Learn: [link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)

In [7]:
# para definir o número de tópicos.
df['categoria'].unique()

array(['livro', 'brinquedo', 'maquiagem', 'game'], dtype=object)

In [8]:
# resposta
from sklearn.decomposition import LatentDirichletAllocation

# modelo
LDA = LatentDirichletAllocation(n_components=4,random_state=42)
LDA.fit(text_vect_train)

LatentDirichletAllocation(n_components=4, random_state=42)

In [9]:
# depois de executar o LDA, podemos ver os resultados executando o código abaixo (considerando que a instancia do modelo seja
# a variável LDA:
# top palavras dos tópicos
terms = vect.get_feature_names_out()

for index, topic in enumerate(LDA.components_):
    terms_comp = zip(terms, topic)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print("THE TOP 15 WORDS FOR TOPIC # "+str(index)+": ")
    #print(sorted_terms)
    print([t[0] for t in sorted_terms])
    print('\n')

THE TOP 15 WORDS FOR TOPIC # 0: 
['produto', 'edição', 'livro', 'páginas', 'ter', 'poder', 'vida', 'mundo', 'novo', 'história', 'anos', '00', 'livros', 'capa', 'origem']


THE TOP 15 WORDS FOR TOPIC # 1: 
['produto', 'cm', 'bonecos', 'kit', 'boneco', 'pronta', 'super', 'ball', 'dragon', '12', 'novo', 'frete', 'personagem', '100', 'vingadores']


THE TOP 15 WORDS FOR TOPIC # 2: 
['pincel', 'cm', 'maquiagem', 'kit', 'cílios', 'cores', 'maleta', 'profissional', '01', 'base', 'matte', 'sombra', 'produto', 'cor', 'the']


THE TOP 15 WORDS FOR TOPIC # 3: 
['mercado', 'produto', 'pagamento', 'prazo', 'frete', 'envio', 'produtos', 'enviar', 'entrega', 'jogo', 'compra', 'ter', 'dias', 'vendedor', 'comprar']




## ToDo 4
Usando o df_test, faça:

1. Vetorize a coluna texto_lemma do df_test

2. Obtenha o tópico a partir da coluna vetorizada para cada exemplo no df_test. Use o método transform do LDA. 

3. Crie uma nova coluna chamada "tópico" no df_test que conterá o tópico de maior probabilidade. (use argmax para recuperar o tópico de maior probabilidade).

In [10]:
# resposta
text_vect_test = vect.transform(df_test.texto_lemma)
results = LDA.transform(text_vect_test)

df_test['topico'] = results.argmax(axis=1)

df_test.head()

,nome,descricao,categoria,texto,texto_lemma,topico
2700,Estojo Duo Iluminador E Bronzer Belle Angel M...,DUO ILUMINADOR E BRONZER BELLE ANGEL - B025O D...,maquiagem,Estojo Duo Iluminador E Bronzer Belle Angel M...,Estojo Duo Iluminador E Bronzer Belle Angel ...,2
1297,Patrulha Canina Carrinhos De Fricção 6 Person...,Kit carrinhos da Patrulha canina Tamanho: Cerc...,brinquedo,Patrulha Canina Carrinhos De Fricção 6 Person...,Patrulha Canina Carrinhos De Fricção 6 Perso...,1
3162,Pokemon Ultra Moon Nintendo 3ds Midia Fisica ...,#Nossos produtos são NOVOS e 100% ORIGINAIS# ...,game,Pokemon Ultra Moon Nintendo 3ds Midia Fisica ...,Pokemon Ultra Moon Nintendo 3ds Midia Fisica...,3
2895,Kite Pincel 12 Unidade,esta alta qualidade Punho de madeira Escova Co...,maquiagem,Kite Pincel 12 Unidade esta alta qualidade ...,Kite Pincel 12 Unidade esta alta qualida...,2
2226,Maleta Grande P/ Maquiagem Profissional Rodin...,FOTOS REAIS DO PRODUTO - ENVIAMOS SUA MALETA C...,maquiagem,Maleta Grande P/ Maquiagem Profissional Rodin...,Maleta Grande P/ Maquiagem Profissional Rodi...,2


## ToDo 5
Use a função built-in map para mapear cada categoria para os seguintes números:

* livro -> 0
* brinquedo -> 1
* maquiagem -> 2
* game -> 3

Crie uma coluna "categ_cod" para armazenar o código de cada categoria

In [11]:
# resposta
df_test['categ_cod'] = df_test.categoria.map(
    {'livro': 0, 'brinquedo': 1, 'maquiagem': 2, 'game': 3 } 
    )

In [13]:
df_test.head()

,nome,descricao,categoria,texto,texto_lemma,topico,categ_cod
2700,Estojo Duo Iluminador E Bronzer Belle Angel M...,DUO ILUMINADOR E BRONZER BELLE ANGEL - B025O D...,maquiagem,Estojo Duo Iluminador E Bronzer Belle Angel M...,Estojo Duo Iluminador E Bronzer Belle Angel ...,2,2
1297,Patrulha Canina Carrinhos De Fricção 6 Person...,Kit carrinhos da Patrulha canina Tamanho: Cerc...,brinquedo,Patrulha Canina Carrinhos De Fricção 6 Person...,Patrulha Canina Carrinhos De Fricção 6 Perso...,1,1
3162,Pokemon Ultra Moon Nintendo 3ds Midia Fisica ...,#Nossos produtos são NOVOS e 100% ORIGINAIS# ...,game,Pokemon Ultra Moon Nintendo 3ds Midia Fisica ...,Pokemon Ultra Moon Nintendo 3ds Midia Fisica...,3,3
2895,Kite Pincel 12 Unidade,esta alta qualidade Punho de madeira Escova Co...,maquiagem,Kite Pincel 12 Unidade esta alta qualidade ...,Kite Pincel 12 Unidade esta alta qualida...,2,2
2226,Maleta Grande P/ Maquiagem Profissional Rodin...,FOTOS REAIS DO PRODUTO - ENVIAMOS SUA MALETA C...,maquiagem,Maleta Grande P/ Maquiagem Profissional Rodin...,Maleta Grande P/ Maquiagem Profissional Rodi...,2,2


## ToDo 6
Obtenha a acurácia usando "categ_cod" como y_true e "topico" como y_pred

In [12]:
# resposta
from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(df_test.categ_cod, df_test.topico))

0.8194285714285714
